In [1]:
import subprocess, threading
from ipywidgets import Output
from IPython.display import display
from faster_whisper_live import LiveWhisper

# —— 前端输出区 —— 
out = Output(layout={"border":"1px solid #ccc","height":"400px","overflow":"auto"})
display(out)

# —— 初始化模型 —— 
model = LiveWhisper("small", device="cpu", compute_type="int8")

# —— 启动 FFmpeg 采麦克风 —— 
ffmpeg_cmd = [
    "ffmpeg",
    "-loglevel", "quiet",
    "-f", "avfoundation",
    "-i", ":1",       # ← 用你的音频设备号
    "-ac", "1",
    "-ar", "16000",
    "-f", "s16le",
    "-"
]
proc = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE)

# —— 后台线程：一边喂流，一边打印 —— 
def run_live():
    with out:
        for seg in model.transcribe(proc.stdout,
                                    decode_audio=False,
                                    vad_filter=True):
            print(f"[{seg.start:.2f}s → {seg.end:.2f}s] {seg.text}")

threading.Thread(target=run_live, daemon=True).start()

Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…